In [1]:
import re
import json
import os
import sys
import random
import numpy as np
import torch

from IPython.display import display, Markdown

from transformers import AutoTokenizer

from patching import *
from plotting import *

/Users/yiranshi/dev/medical-gpt-interpretability/ENV/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_name = "dmis-lab/meerkat-7b-v1.0"
tokenizer = AutoTokenizer.from_pretrained(model_name)

## Plots Demo

In [3]:
demo_tensor_path = "../data/patching_results_5.pt"
# Load the patching results tensor
if os.path.exists(demo_tensor_path):
    patching_results = torch.load(demo_tensor_path, map_location=torch.device("cpu"))
    print(patching_results.shape)
    
    # Example: Display some basic statistics about the tensor
    print(f"Min value: {patching_results.min().item()}")
    print(f"Max value: {patching_results.max().item()}")
    print(f"Mean value: {patching_results.mean().item()}")
    
    # Optionally visualize the tensor (if it's a 2D tensor suitable for heatmap)
    # Load the corresponding tokens for visualization
    # This assumes you have the tokens saved or can reconstruct them
    # For demonstration, we'll create dummy tokens
    dummy_tokens = torch.ones((1, patching_results.shape[1]), dtype=torch.long)
    
    # Plot the patching heatmap
    plot_patching_heatmap(
        patching_results,
        dummy_tokens,
        tokenizer,
        answer="A",  # Replace with actual answer if available
        filepath="../results/demo_patching_heatmap.pdf"
    )

    plot_average_patching_heatmap(
        patching_results,
        0,
        37,
        filepath="../results/demo_average_patching_heatmap.pdf"
    )


torch.Size([32, 471])
Min value: -0.66796875
Max value: 1.3359375
Mean value: 0.11396066844463348


KeyError: 'first patient token'

## Tokenizer Experiment

In [6]:
model_name = "dmis-lab/meerkat-7b-v1.0"
tokenizer = AutoTokenizer.from_pretrained(model_name)

with open("../data/meerkat-7b-v1.0_medqa-original_results.json", "r") as f:
    baseline_data = json.load(f)

max_tokens = 1000

In [7]:
id = 5
prompt = baseline_data[id]["generated_response"]
display(Markdown(prompt))

The following is a multiple-choice question about medical knowledge. Solve this in a step-by-step fashion, starting by summarizing the available information. Output a single option from the given options as the final answer. You are strongly required to follow the specified output format; conclude your response with the phrase "the answer is ([option_id]) [answer_string]".

USER: A 23-year-old woman comes to the physician because she is embarrassed about the appearance of her nails. She has no history of serious illness and takes no medications. She appears well. A photograph of the nails is shown. Which of the following additional findings is most likely in this patient? (A) Silvery plaques on extensor surfaces (B) Flesh-colored papules in the lumbosacral region (C) Erosions of the dental enamel (D) Holosystolic murmur at the left lower sternal border ASSISTANT: 

The prompt does not provide an image, but based on the context of the question which is about the appearance of the patient's nails, and the fact that she is embarrassed by it, we can infer that the patient might be presenting with a cosmetic or dermatological issue with her nails. We are to choose the most likely additional finding among the four options provided.

(A) Silvery plaques on extensor surfaces: This finding is suggestive of psoriasis, which can present with nail changes such as pitting or onycholysis (separation of the nail from the nail bed). Psoriasis also causes silvery plaques, typically on the extensor surfaces such as elbows and knees, which could be consistent with a cosmetic concern the patient might have.

(B) Flesh-colored papules in the lumbosacral region: This description is consistent with conditions like warts (human papillomavirus infection), which can be found anywhere on the body but are more typical in areas of skin contact. They are less likely to be related to abnormalities of the nails.

(C) Erosions of the dental enamel: This could be found with conditions such as bulimia or acid reflux, where the teeth might be eroded due to repeated exposure to stomach acid. However, there is nothing in the question stem to suggest that this patient has a history of a condition that would cause dental enamel erosion.

(D) Holosystolic murmur at the left lower sternal border: A holosystolic murmur is related to a heart condition, like ventricular septal defect or mitral regurgitation. While heart murmurs can be associated with various systemic conditions, there is no direct link provided in the question between the patient’s nail appearance and a heart murmur.

Based on the information given and the process of elimination, the most likely additional finding that could be associated with a cosmetic concern about the nails is (A) Silvery plaques on extensor surfaces, which could be a sign of psoriasis and might be accompanied by nail changes. Therefore, the answer is (A) Silvery plaques on extensor surfaces.</s>

In [8]:
prompt = setup_prompt(prompt)
display(Markdown(prompt))

 

The prompt does not provide an image, but based on the context of the question which is about the appearance of the patient's nails, and the fact that she is embarrassed by it, we can infer that the patient might be presenting with a cosmetic or dermatological issue with her nails. We are to choose the most likely additional finding among the four options provided.

(A) Silvery plaques on extensor surfaces: This finding is suggestive of psoriasis, which can present with nail changes such as pitting or onycholysis (separation of the nail from the nail bed). Psoriasis also causes silvery plaques, typically on the extensor surfaces such as elbows and knees, which could be consistent with a cosmetic concern the patient might have.

(B) Flesh-colored papules in the lumbosacral region: This description is consistent with conditions like warts (human papillomavirus infection), which can be found anywhere on the body but are more typical in areas of skin contact. They are less likely to be related to abnormalities of the nails.

(C) Erosions of the dental enamel: This could be found with conditions such as bulimia or acid reflux, where the teeth might be eroded due to repeated exposure to stomach acid. However, there is nothing in the question stem to suggest that this patient has a history of a condition that would cause dental enamel erosion.

(D) Holosystolic murmur at the left lower sternal border: A holosystolic murmur is related to a heart condition, like ventricular septal defect or mitral regurgitation. While heart murmurs can be associated with various systemic conditions, there is no direct link provided in the question between the patient’s nail appearance and a heart murmur.

Based on the information given and the process of elimination, the most likely additional finding that could be associated with a cosmetic concern about the nails is (A) Silvery plaques on extensor surfaces, which could be a sign of psoriasis and might be accompanied by nail changes. Therefore, the answer is (

In [9]:
patient_gender_filter = PatientInfoFilter()
patient_gender = patient_gender_filter.extract_gender(prompt)
counterfactual_prompt = generate_counterfactual_patient_info(prompt, patient_gender)
display(Markdown(counterfactual_prompt))

 

The prompt does not provide an image, but based on the context of the question which is about the appearance of the patient's nails, and the fact that he is embarrassed by it, we can infer that the patient might be presenting with a cosmetic or dermatological issue with his nails. We are to choose the most likely additional finding among the four options provided.

(A) Silvery plaques on extensor surfaces: This finding is suggestive of psoriasis, which can present with nail changes such as pitting or onycholysis (separation of the nail from the nail bed). Psoriasis also causes silvery plaques, typically on the extensor surfaces such as elbows and knees, which could be consistent with a cosmetic concern the patient might have.

(B) Flesh-colored papules in the lumbosacral region: This description is consistent with conditions like warts (human papillomavirus infection), which can be found anywhere on the body but are more typical in areas of skin contact. They are less likely to be related to abnormalities of the nails.

(C) Erosions of the dental enamel: This could be found with conditions such as bulimia or acid reflux, where the teeth might be eroded due to repeated exposure to stomach acid. However, there is nothing in the question stem to suggest that this patient has a history of a condition that would cause dental enamel erosion.

(D) Holosystolic murmur at the left lower sternal border: A holosystolic murmur is related to a heart condition, like ventricular septal defect or mitral regurgitation. While heart murmurs can be associated with various systemic conditions, there is no direct link provided in the question between the patient’s nail appearance and a heart murmur.

Based on the information given and the process of elimination, the most likely additional finding that could be associated with a cosmetic concern about the nails is (A) Silvery plaques on extensor surfaces, which could be a sign of psoriasis and might be accompanied by nail changes. Therefore, the answer is (

In [7]:
clean_tokens = tokenizer.encode(prompt, return_tensors="pt")
corrupted_tokens = tokenizer.encode(counterfactual_prompt, return_tensors="pt")

print(len(clean_tokens[0]))
print(len(corrupted_tokens[0]))

397
397


In [3]:
pronouns = ["he", "his", "him", "she", "her", "hers", "man", "male", "woman", "female", "hers"]
token_lengths = []

for pronoun in pronouns:
    token_lengths.append(len(tokenizer.encode(pronoun)))

token_lengths

[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]

## Results

In [2]:
output_file = "meerkat-7b-v1.0_results.json"
invalid_file = "meerkat-7b-v1.0_invalid_ids.json"

with open(output_file, "r") as f:
    data = json.load(f)

with open(invalid_file, "r") as f:
    invalid_ids = json.load(f)

In [3]:
id = 467
output = data[406]["generated_response"]
actual_id = data[406]["id"]
print(f"id: {actual_id}")
display(Markdown(output))

id: test-00406


The following is a multiple-choice question about medical knowledge. Solve this in a step-by-step fashion, starting by summarizing the available information. Output a single option from the given options as the final answer. You are strongly required to follow the specified output format; conclude your response with the phrase "the answer is ([option_id]) [answer_string]".

USER: A 54-year-old man presents with the acute onset of seizures and hallucinations. The patient’s wife says that he had been on a camping trip a few months earlier to study bats but otherwise has not traveled recently. Past medical history is significant for hypertension, managed medically with hydralazine and enalapril. The patient’s condition rapidly deteriorates, and he passes away. An autopsy is performed and a histologic stained section of the brain is shown in the image. Which of the following receptors are targeted by the pathogen most likely responsible for this patient’s condition? (A) CCR5 (B) Acetylcholine receptor (C) ICAM-1 (D) GABA ASSISTANT: 
The key pieces of information from the question are as follows: 
- The patient is a 54-year-old man.
- He presented with acute seizures and hallucinations.
- He has a history of hypertension.
- He went on a camping trip to study bats a few months earlier.
- He has not traveled recently.
- His condition rapidly deteriorated and he passed away.
- An autopsy is performed suggesting a post-mortem diagnosis.

Given the history of the patient studying bats and the acute neurological symptoms, a viral etiology is strongly suggested, particularly a rabies virus infection. Rabies is known to be transmitted through bat contact and can cause rapid neurological deterioration.

Rabies virus primarily infects neurons and utilizes the nicotinic acetylcholine receptors (nAChR) at the neuromuscular junction as a entry point into the central nervous system. After the virus enters the CNS, it causes encephalitis and the described symptoms.

Let's consider the receptor options provided:
(A) CCR5 - This is a chemokine receptor used by HIV to enter cells, not relevant to rabies.
(B) Acetylcholine receptor - This is the correct receptor targeted by rabies virus.
(C) ICAM-1 - This is an intercellular adhesion molecule involved in immune responses, not the primary receptor for rabies virus.
(D) GABA - This is a neurotransmitter in the brain, not a receptor targeted by rabies virus.

Based on this information and the clinical presentation, the answer is that the pathogen most likely responsible for this patient’s condition targets the acetylcholine receptor. Therefore, the answer is (B) Acetylcholine receptor.</s>

In [9]:
# extract the answer from the output
answer = re.search(r'(?:therefore,\s*)?the answer is.*?\(([A-D])\)', output, re.IGNORECASE)
if answer:
    print(answer.group(1))
else:
    print("No answer found")

B


In [7]:
# Let's find all outputs that contain the phrase "a 54-year-old man" (case insensitive)
pattern = r'a \d+[-\s]year[-\s]old (?:man|woman)'
matching_ids = []

for item in data:
    if re.search(pattern, item["generated_response"], re.IGNORECASE):
        matching_ids.append(item["id"])

len(matching_ids)


737